In [ ]:
def feed_students(excel_file_name):
    import pandas as pd
    import mysql.connector
    from mysql.connector import Error

    df = pd.read_excel(excel_file_name, engine="openpyxl")
    df.columns = ['name', 'age', 'university', 'college', 'department', 'year_of_study', 'email', 'phone']

    connection = None

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            for _, row in df.iterrows():
                sql = """
                INSERT INTO students (name, age, university, college, department, year_of_study, email, phone)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    str(row['name']),
                    int(row['age']),
                    str(row['university']),
                    str(row['college']),
                    str(row['department']),
                    str(row['year_of_study']),
                    str(row['email']),
                    str(row['phone'])
                )
                cursor.execute(sql, values)
                student_id = cursor.lastrowid

                # Create daily progress table with 3 task types
                cursor.execute(f"""
                    CREATE TABLE IF NOT EXISTS daily_progress_{student_id} (
                        id INT PRIMARY KEY AUTO_INCREMENT,
                        student_id INT,
                        date DATE,
                        research_tasks_done INT,
                        research_tasks_total INT,
                        english_tasks_done INT,
                        english_tasks_total INT,
                        other_tasks_done INT,
                        other_tasks_total INT,
                        total_tasks_done INT,
                        total_tasks_total INT,
                        prcentage_done DOUBLE,
                        FOREIGN KEY (student_id) REFERENCES students(student_id)
                    )
                """)

                # Create weekly progress table with 3 task types
                cursor.execute(f"""
                    CREATE TABLE IF NOT EXISTS weekly_progress_{student_id} (
                        id INT PRIMARY KEY AUTO_INCREMENT,
                        student_id INT,
                        week_start DATE,
                        research_tasks_done INT,
                        research_tasks_total INT,
                        english_tasks_done INT,
                        english_tasks_total INT,
                        other_tasks_done INT,
                        other_tasks_total INT,
                        total_tasks_done INT,
                        total_tasks_total INT,
                        prcentage_done DOUBLE,
                        stat_of_week BOOLEAN DEFAULT true,
                        FOREIGN KEY (student_id) REFERENCES students(student_id)
                    )
                """)

            connection.commit()
            print("✅ Student data and updated progress tables with task types have been successfully created.")

    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()



#______________________________________________________________________________

def add_student(student_data):
    """
    Adds a student to the students table and creates associated daily and weekly progress tables.
    Supports multiple task types: research, english, other.
    """
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # Insert student
            sql = """
            INSERT INTO students (name, age, university, college, department, year_of_study, email, phone)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                student_data['name'],
                student_data['age'],
                student_data['university'],
                student_data['college'],
                student_data['department'],
                student_data['year_of_study'],
                student_data['email'],
                student_data['phone']
            )
            cursor.execute(sql, values)
            student_id = cursor.lastrowid

            # Create daily progress table with task types
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS daily_progress_{student_id} (
                    id INT PRIMARY KEY AUTO_INCREMENT,
                    student_id INT,
                    date DATE,
                    research_tasks_done INT,
                    research_tasks_total INT,
                    english_tasks_done INT,
                    english_tasks_total INT,
                    other_tasks_done INT,
                    other_tasks_total INT,
                    total_tasks_done INT,
                    total_tasks_total INT,
                    prcentage_done DOUBLE,
                    FOREIGN KEY (student_id) REFERENCES students(student_id)
                )
            """)

            # Create weekly progress table with task types
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS weekly_progress_{student_id} (
                    id INT PRIMARY KEY AUTO_INCREMENT,
                    student_id INT,
                    week_start DATE,
                    research_tasks_done INT,
                    research_tasks_total INT,
                    english_tasks_done INT,
                    english_tasks_total INT,
                    other_tasks_done INT,
                    other_tasks_total INT,
                    total_tasks_done INT,
                    total_tasks_total INT,
                    prcentage_done DOUBLE,
                    stat_of_week BOOLEAN DEFAULT true,
                    FOREIGN KEY (student_id) REFERENCES students(student_id)
                )
            """)

            connection.commit()
            print(f"✅ Student added with ID {student_id} and progress tables created.")

    except Error as e:
        print(f"❌ Error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()


#______________________________________________________________________________

def remove_student(student_id):
    """
    Removes a student from the students table and drops their associated progress tables.
    """
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # Drop progress tables
            cursor.execute(f"DROP TABLE IF EXISTS daily_progress_{student_id}")
            cursor.execute(f"DROP TABLE IF EXISTS weekly_progress_{student_id}")

            # Delete student
            cursor.execute("DELETE FROM students WHERE student_id = %s", (student_id,))

            connection.commit()
            print(f"🗑️ Student with ID {student_id} and associated progress tables removed.")

    except Error as e:
        print(f"❌ Error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()




#______________________________________________________________________________

def update_student_field(student_id, column_name, new_value):
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',       # 🔁 Replace with your MySQL username
            password='power2020',   # 🔁 Replace with your MySQL password
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # Build dynamic SQL query safely
            sql = f"UPDATE students SET {column_name} = %s WHERE student_id = %s"
            cursor.execute(sql, (new_value, student_id))
            connection.commit()
            print(f"✅ Updated {column_name} for student ID {student_id}.")

    except Error as e:
        print(f"❌ Error while updating student: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()



#______________________________________________________________________________


def generate_demo_progress_data():
    import mysql.connector
    from mysql.connector import Error
    from datetime import datetime, timedelta
    import random
    
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)

            # Fetch all students
            cursor.execute("SELECT student_id FROM students")
            students = cursor.fetchall()

            for student in students:
                student_id = student['student_id']

                # Generate 50 consecutive daily progress entries
                start_date = datetime.now().date() - timedelta(days=49)
                for i in range(50):
                    date = start_date + timedelta(days=i)

                    # Generate random task counts for each type
                    r_total = random.randint(1, 4)
                    r_done = random.randint(0, r_total)

                    e_total = random.randint(1, 3)
                    e_done = random.randint(0, e_total)

                    o_total = random.randint(1, 3)
                    o_done = random.randint(0, o_total)

                    total_done = r_done + e_done + o_done
                    total_total = r_total + e_total + o_total
                    percentage_done = round((total_done / total_total) * 100, 2) if total_total else 0.0

                    cursor.execute(f"""
                        INSERT INTO daily_progress_{student_id} 
                        (student_id, date,
                         research_tasks_done, research_tasks_total,
                         english_tasks_done, english_tasks_total,
                         other_tasks_done, other_tasks_total,
                         total_tasks_done, total_tasks_total, prcentage_done)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (
                        student_id, date,
                        r_done, r_total,
                        e_done, e_total,
                        o_done, o_total,
                        total_done, total_total, percentage_done
                    ))

            connection.commit()
            print("✅ 50 days of structured daily demo progress data inserted for all students.")

    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

#______________________________________________________________________________

def add_progress_row(student_id, table_type, data):
    """
    Adds a single row to the daily or weekly progress table for a given student.
    table_type: 'daily' or 'weekly'
    data: dictionary containing task counts for research, english, and other.
    """
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            # Calculate totals
            total_done = data['research_tasks_done'] + data['english_tasks_done'] + data['other_tasks_done']
            total_total = data['research_tasks_total'] + data['english_tasks_total'] + data['other_tasks_total']
            prcentage_done = round((total_done / total_total) * 100, 2) if total_total else 0.0

            if table_type == 'daily':
                sql = f"""
                    INSERT INTO daily_progress_{student_id} 
                    (student_id, date,
                     research_tasks_done, research_tasks_total,
                     english_tasks_done, english_tasks_total,
                     other_tasks_done, other_tasks_total,
                     total_tasks_done, total_tasks_total, prcentage_done)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    student_id, data['date'],
                    data['research_tasks_done'], data['research_tasks_total'],
                    data['english_tasks_done'], data['english_tasks_total'],
                    data['other_tasks_done'], data['other_tasks_total'],
                    total_done, total_total, prcentage_done
                )

            elif table_type == 'weekly':
                stat_of_week = 1 if prcentage_done >= 75 else 0
                sql = f"""
                    INSERT INTO weekly_progress_{student_id} 
                    (student_id, week_start,
                     research_tasks_done, research_tasks_total,
                     english_tasks_done, english_tasks_total,
                     other_tasks_done, other_tasks_total,
                     total_tasks_done, total_tasks_total, prcentage_done, stat_of_week)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                values = (
                    student_id, data['week_start'],
                    data['research_tasks_done'], data['research_tasks_total'],
                    data['english_tasks_done'], data['english_tasks_total'],
                    data['other_tasks_done'], data['other_tasks_total'],
                    total_done, total_total, prcentage_done, stat_of_week
                )

            else:
                print("❌ Invalid table type. Use 'daily' or 'weekly'.")
                return

            cursor.execute(sql, values)
            connection.commit()
            print(f"✅ Row added to {table_type}_progress_{student_id}.")

    except Error as e:
        print(f"❌ Error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()



#______________________________________________________________________________

def remove_progress_row(student_id, table_type, row_id):
    """
    Removes a row from the daily or weekly progress table for a given student.
    table_type: 'daily' or 'weekly'
    row_id: ID of the row to delete
    """
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            if table_type in ['daily', 'weekly']:
                table_name = f"{table_type}_progress_{student_id}"
                sql = f"DELETE FROM {table_name} WHERE id = %s"
                cursor.execute(sql, (row_id,))
                connection.commit()
                print(f"✅ Row with ID {row_id} removed from {table_name}.")
            else:
                print("❌ Invalid table type. Use 'daily' or 'weekly'.")

    except Error as e:
        print(f"❌ Error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()



#______________________________________________________________________________

def backfill_weekly_progress():
    import mysql.connector
    from mysql.connector import Error
    from datetime import timedelta
    
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)

            # Get all students
            cursor.execute("SELECT student_id FROM students")
            students = cursor.fetchall()

            for student in students:
                student_id = student['student_id']
                daily_table = f"daily_progress_{student_id}"
                weekly_table = f"weekly_progress_{student_id}"

                try:
                    # Fetch all daily progress entries
                    cursor.execute(f"""
                        SELECT date,
                               research_tasks_done, research_tasks_total,
                               english_tasks_done, english_tasks_total,
                               other_tasks_done, other_tasks_total
                        FROM {daily_table}
                        ORDER BY date
                    """)
                    daily_entries = cursor.fetchall()

                    # Group by week starting Saturday
                    weekly_data = {}
                    for entry in daily_entries:
                        date = entry['date']
                        week_start = date - timedelta(days=(date.weekday() + 2) % 7)

                        if week_start not in weekly_data:
                            weekly_data[week_start] = {
                                'research_done': 0, 'research_total': 0,
                                'english_done': 0, 'english_total': 0,
                                'other_done': 0, 'other_total': 0
                            }

                        weekly_data[week_start]['research_done'] += entry['research_tasks_done']
                        weekly_data[week_start]['research_total'] += entry['research_tasks_total']
                        weekly_data[week_start]['english_done'] += entry['english_tasks_done']
                        weekly_data[week_start]['english_total'] += entry['english_tasks_total']
                        weekly_data[week_start]['other_done'] += entry['other_tasks_done']
                        weekly_data[week_start]['other_total'] += entry['other_tasks_total']

                    # Insert into weekly table
                    for week_start, data in weekly_data.items():
                        total_done = data['research_done'] + data['english_done'] + data['other_done']
                        total_total = data['research_total'] + data['english_total'] + data['other_total']
                        percentage = round((total_done / total_total) * 100, 2) if total_total else 0.0
                        stat_of_week = 1 if percentage >= 75 else 0

                        # Avoid duplicates
                        cursor.execute(f"""
                            SELECT COUNT(*) AS count FROM {weekly_table}
                            WHERE week_start = %s
                        """, (week_start,))
                        exists = cursor.fetchone()['count']

                        if exists == 0:
                            cursor.execute(f"""
                                INSERT INTO {weekly_table}
                                (student_id, week_start,
                                 research_tasks_done, research_tasks_total,
                                 english_tasks_done, english_tasks_total,
                                 other_tasks_done, other_tasks_total,
                                 total_tasks_done, total_tasks_total,
                                 prcentage_done, stat_of_week)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                            """, (
                                student_id, week_start,
                                data['research_done'], data['research_total'],
                                data['english_done'], data['english_total'],
                                data['other_done'], data['other_total'],
                                total_done, total_total, percentage, stat_of_week
                            ))

                    print(f"✅ Weekly progress backfilled for student ID {student_id}.")

                except Error as e:
                    print(f"⚠️ Could not process student {student_id}: {e}")

            connection.commit()

    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()



#______________________________________________________________________________


def update_progress_column(student_id, table_type, row_id, column_name, new_value):
    """
    Updates a specific column in a progress row for a given student.
    
    Parameters:
        student_id (int): ID of the student
        table_type (str): 'daily' or 'weekly'
        row_id (int): ID of the row to update
        column_name (str): Name of the column to update
        new_value: New value to set
    """
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            if table_type not in ['daily', 'weekly']:
                print("❌ Invalid table type. Use 'daily' or 'weekly'.")
                return

            table_name = f"{table_type}_progress_{student_id}"
            sql = f"UPDATE {table_name} SET {column_name} = %s WHERE id = %s"
            cursor.execute(sql, (new_value, row_id))
            connection.commit()
            print(f"✅ Updated column '{column_name}' in row ID {row_id} of {table_name}.")

    except Error as e:
        print(f"❌ Error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

#______________________________________________________________________________


def count_weekly_stat_of_week_zero_and_update_marks():
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            cursor.execute("SELECT student_id, name FROM students")
            students = cursor.fetchall()

            for student in students:
                student_id = student['student_id']
                table_name = f"weekly_progress_{student_id}"

                try:
                    cursor.execute(f"SELECT COUNT(*) AS zero_count FROM {table_name} WHERE stat_of_week = 0")
                    result = cursor.fetchone()
                    zero_count = result['zero_count']

                    cursor.execute("UPDATE students SET marks = %s WHERE student_id = %s", (zero_count, student_id))
                    print(f"Student: {student_id}, stat_of_week = 0 count: {zero_count} (marks updated)")

                except Error as e:
                    print(f"⚠️ Could not query or update for table {table_name}: {e}")

            connection.commit()

    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

#______________________________________________________________________________


def evaluate_weekly_progress_and_update_membership():
    import mysql.connector
    from mysql.connector import Error

    def update_student_field(student_id, column_name, new_value):
        try:
            conn = mysql.connector.connect(
                host='localhost',
                user='root',
                password='power2020',
                database='student_tracker',
                charset='utf8mb4'
            )
            if conn.is_connected():
                cur = conn.cursor()
                cur.execute(f"UPDATE students SET {column_name} = %s WHERE student_id = %s", (new_value, student_id))
                conn.commit()
                cur.close()
                conn.close()
        except Error as e:
            print(f"❌ Error while updating student: {e}")

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            cursor.execute("SELECT student_id FROM students")
            students = cursor.fetchall()

            for student in students:
                student_id = student['student_id']
                table_name = f"weekly_progress_{student_id}"

                try:
                    cursor.execute(f"SELECT stat_of_week FROM {table_name} ORDER BY week_start")
                    rows = cursor.fetchall()
                    zero_count = sum(1 for row in rows if row['stat_of_week'] == 0)

                    consecutive_zeros = any(
                        rows[i]['stat_of_week'] == 0 and rows[i + 1]['stat_of_week'] == 0
                        for i in range(len(rows) - 1)
                    )

                    if zero_count >= 3 or consecutive_zeros:
                        update_student_field(student_id, 'member_state', 0)

                except Error as e:
                    print(f"⚠️ Could not process table {table_name}: {e}")

    except Error as e:
        print(f"❌ Error while connecting to MySQL: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

#______________________________________________________________________________


def validate_stat_of_week_and_adjust_marks():
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)
            cursor.execute("SELECT student_id, name, marks FROM students WHERE marks > 0")
            students = cursor.fetchall()

            for student in students:
                student_id = student['student_id']
                marks = student['marks']
                table_name = f"weekly_progress_{student_id}"

                try:
                    cursor.execute(f"SELECT id, stat_of_week FROM {table_name} ORDER BY week_start")
                    progress_rows = cursor.fetchall()

                    updated_rows = 0
                    for i in range(len(progress_rows)):
                        if progress_rows[i]['stat_of_week'] == 0:
                            if i + 2 < len(progress_rows):
                                if (progress_rows[i + 1]['stat_of_week'] == 1 and
                                    progress_rows[i + 2]['stat_of_week'] == 1):
                                    row_id = progress_rows[i]['id']
                                    cursor.execute(f"""
                                        UPDATE {table_name}
                                        SET stat_of_week = 1
                                        WHERE id = %s
                                    """, (row_id,))
                                    updated_rows += 1

                    if updated_rows > 0:
                        new_marks = max(0, marks - updated_rows)
                        cursor.execute("""
                            UPDATE students
                            SET marks = %s
                            WHERE student_id = %s
                        """, (new_marks, student_id))
                        print(f"✅ (ID: {student_id}) - {updated_rows} stat_of_week flipped, marks updated to {new_marks}")

                except Error as e:
                    print(f"⚠️ Error processing {table_name}: {e}")

            connection.commit()

    except Error as e:
        print(f"❌ Connection error: {e}")

    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

#______________________________________________________________________________


import pandas as pd
import mysql.connector
from mysql.connector import Error
from datetime import datetime

def add_daily_progress_from_excel(excel_file, research_total, english_total, other_total):
    try:
        df = pd.read_excel(excel_file, engine='openpyxl')
        df.columns = ['student_id', 'research_tasks_done', 'english_tasks_done', 'other_tasks_done']

        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='power2020',
            database='student_tracker',
            charset='utf8mb4'
        )

        if connection.is_connected():
            cursor = connection.cursor()

            for _, row in df.iterrows():
                student_id = int(row['student_id'])
                r_done = int(row['research_tasks_done'])
                e_done = int(row['english_tasks_done'])
                o_done = int(row['other_tasks_done'])

                total_done = r_done + e_done + o_done
                total_total = research_total + english_total + other_total
                percentage_done = round((total_done / total_total) * 100, 2) if total_total else 0.0

                cursor.execute(f"""
                    INSERT INTO daily_progress_{student_id}
                    (student_id, date,
                     research_tasks_done, research_tasks_total,
                     english_tasks_done, english_tasks_total,
                     other_tasks_done, other_tasks_total,
                     total_tasks_done, total_tasks_total, prcentage_done)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    student_id, datetime.now().date(),
                    r_done, research_total,
                    e_done, english_total,
                    o_done, other_total,
                    total_done, total_total, percentage_done
                ))

            connection.commit()
            print("✅ Daily progress rows inserted successfully for all students.")

    except Error as e:
        print(f"❌ Database error: {e}")
    except Exception as ex:
        print(f"❌ General error: {ex}")
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
